# 리뷰 기반 게임 검색

In [1]:
import os
import time
import pickle
import joblib
import sys

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer #CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk

from nltk.corpus import stopwords
from collections import Counter

# 추가한 라이브러리
import re
import joblib

In [2]:
# read Dataset of 크롤링

with open("preprocessing/game_pkl/game_df_b150.pkl","rb") as fr:
    game_df = pickle.load(fr)

In [3]:
game_df.head()

,platform_2,platform,title,publisher,static,detail,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
0,1,DS,Ghost Trick: Phantom Detective,Capcom,"{'user': {'positive': 1552, 'mixed': 12, 'nega...",{'summary': 'Ghost Trick is a story of mystery...,"{'score': 10, 'review': 'Easily one of the bes...","{'score': 9, 'review': 'Hands down, one of the...","{'score': 10, 'review': 'Innovative and addict...","{'score': 10, 'review': 'Phenomenal. This game...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PSP,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,NIS America,"{'user': {'positive': 1038, 'mixed': 3, 'negat...",{'summary': 'Known as ZettaiHero Keikakuin Jap...,"{'score': 8, 'review': 'Probably the best rogu...","{'score': 8, 'review': 'If you aren't familiar...","{'score': 10, 'review': 'NIS has always writte...","{'score': 9, 'review': 'It is really amazing t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Xbox Series X,MADiSON,Perp Games,"{'user': {'positive': 97, 'mixed': 1, 'negativ...",{'summary': 'What would you do if you woke up ...,"{'score': 1, 'review': 'It's a very short game...","{'score': 2, 'review': 'The game is very poorl...","{'score': 10, 'review': 'Best horror experienc...","{'score': 10, 'review': 'An Argentine game tha...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,PC,Atelier Sophie 2: The Alchemist of the Mysteri...,Koei Tecmo Games,"{'user': {'positive': 54, 'mixed': 1, 'negativ...",{'summary': 'A mysterious story of a mysteriou...,"{'score': 10, 'review': 'The game turned out w...","{'score': 10, 'review': 'I finished today Atel...","{'score': 10, 'review': 'Wow! Atelier Sophie 2...","{'score': 0, 'review': 'The game is good, ther...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Switch,Dynasty Warriors 9 Empires,Koei Tecmo Games,"{'user': {'positive': 59, 'mixed': 0, 'negativ...","{'summary': 'Empires, a unique ""Romance of the...","{'score': 10, 'review': 'I love this game so m...","{'score': 10, 'review': 'Totally entertained f...","{'score': 10, 'review': 'I really wasn't expec...","{'score': 0, 'review': 'bad graphics, the soun...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print('''리뷰 유사도 분석을 통한 머신러닝 모델입니다.\n
평소 즐겨 하는 platform 을 선택하세요
PC      : 0
Console : 1

'''
     )

while True:
    
    platform = int(input())
    if platform == 1:
        print('Console game을 선택하셨습니다.\n게임 목록입니다.')
        print(game_df[game_df['platform_2'] == platform]['title'])
        
        break
        
    elif platform == 0:
        print('PC game을 선택하셨습니다.\n게임 목록입니다.')
        print(game_df[game_df['platform_2'] == platform]['title'])
        
        break
    
    else:
        print('올바르게 입력하세요')

리뷰 유사도 분석을 통한 머신러닝 모델입니다.

평소 즐겨 하는 platform 을 선택하세요
PC      : 0
Console : 1


1
Console game을 선택하셨습니다.
게임 목록입니다.
0                         Ghost Trick: Phantom Detective
1            Z.H.P. Unlosing Ranger vs Darkdeath Evilman
2                                                MADiSON
4                             Dynasty Warriors 9 Empires
7                     Fatal Frame: Maiden of Black Water
                             ...                        
145                                       Super Mario 64
146                                             Thymesia
147                                Touken Ranbu Warriors
148    Atelier Sophie 2: The Alchemist of the Mysteri...
149    Atelier Sophie 2: The Alchemist of the Mysteri...
Name: title, Length: 104, dtype: object


In [5]:
#불용어 사전
stop_words = stopwords.words("english")

#불용어 추가
stop_words.append(',')
stop_words.append('(')
stop_words.append(')')
stop_words.append('.')
stop_words.append('!')
stop_words.append("'")
stop_words.append("`")
stop_words.append('"')

# 불용어 삭제
#  stop_words.append('\\')

In [6]:
#모든 리뷰 꺼내며 불용어 처리
b_games_review = []
for k in range(len(game_df)):   
    game_review = ''
    for i in range(100):
    
        try:
            game_review = game_review + game_df.iloc[k][i]['review']

        except:
            pass
    
    sentence = game_review
    
    #문장 토큰화
    tokens = nltk.word_tokenize(sentence)
    
    # 불용어 처리
    result = []
    for token in tokens:
        if token.lower() not in stop_words:
            result.append(token.lower())
    
    # 문장 재봉합
    game_review = ''
    for k in result:
        game_review = game_review + ' ' + k

            
    b_games_review.append(game_review)
    

#b_games_review[0]

In [7]:
# ※정규식 필터링 추가

review_word_pattern = re.compile(r'[A-Za-z]+')

# 복잡해 보이지만 로직은 별거 없고
# b_~ 리스트를 순회하며 각 문자열을 정규식 적용하여 단어 분리
# 그 후 길이가 3 이상인 단어만 뽑아 묶은 문자열로 교체
b_games_review = [' '.join([word if len(word) > 2 else '' \
                            for word in re.findall(review_word_pattern, review)]) \
                  for review in b_games_review]

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(b_games_review)

In [9]:
# pd.DataFrame(cosine_similarity(X))

In [10]:
# 유사도가 일정치를 넘는 결과 보기

# cos_array = cosine_similarity(X)
# threshold = 0.6

# for i in range(50):
#     for j in range(i+1, 50):
#         if cos_array[i][j] > threshold:
#             print(game_df.at[i, 'title'], game_df.at[i, 'platform'], '||',game_df.at[j, 'title'], game_df.at[j, 'platform'])

In [11]:
print('해보고 싶은 Game의 번호를 선택하세요')

whlie_n = 0
while True:
    
    game_index = int(input())

    for k in game_df[game_df['platform_2'] == platform]['title'].index:
        if k == game_index:
            whlie_n = 1
    
    if whlie_n == 1:
        break
    else:
        print('없는 Game의 번호입니다. 다시 입력해주세요')
        
print('선택하신 게임의 정보입니다.\n\n'+ game_df['title'][game_index]+ '\n' +
      
'******************************\n'
'Platform : ' + game_df['platform'][game_index] + '\n'
'user   Score ' + str(int(game_df['static'][game_index]['user']['mean']) * 10) + '/100\n'
'static Score ' + str(game_df['static'][game_index]['critic']['mean'])+ '/100\n'
'**********게임설명*************\n' + game_df['detail'][game_index]['summary']
)

해보고 싶은 Game의 번호를 선택하세요
1
선택하신 게임의 정보입니다.

Z.H.P. Unlosing Ranger vs Darkdeath Evilman
******************************
Platform : PSP
user   Score 90/100
static Score 81/100
**********게임설명*************
Known as ZettaiHero Keikakuin Japan, Z.H.P. is the only turn based strategy RPG that allows you to create your own, one and only super hero to fight vicious(?) villains. Players can create a super hero literally from head to toe. If you wish to use drills instead of hands or wheels instead of legs you can do so by simply swapping out your body parts. Battle maps are all randomly generated for mad replay value and there are tons of character and secret base customization. Z.H.P. gives you the chance to relive your childhood fantasy of making your own superhero and superhero base.


In [12]:
pd.DataFrame(cosine_similarity(X)) #수정중

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1.000000,0.233474,0.176895,0.246269,0.245338,0.213269,0.058540,0.173699,0.146191,0.343399,...,0.414374,0.331096,0.265466,0.388480,0.247838,0.350861,0.213027,0.173260,0.191958,0.115457
1,0.233474,1.000000,0.114606,0.138112,0.143486,0.127176,0.033123,0.086245,0.087929,0.259727,...,0.213797,0.198649,0.146714,0.207379,0.165921,0.206495,0.141947,0.098200,0.106107,0.057024
2,0.176895,0.114606,1.000000,0.119485,0.115863,0.109615,0.011963,0.070522,0.096914,0.149209,...,0.191613,0.147222,0.121435,0.161464,0.125451,0.170628,0.118172,0.080181,0.081702,0.041731
3,0.246269,0.138112,0.119485,1.000000,0.178267,0.148095,0.021907,0.086089,0.131543,0.217752,...,0.258992,0.208211,0.169310,0.226944,0.188143,0.233341,0.199036,0.129646,0.358112,0.395254
4,0.245338,0.143486,0.115863,0.178267,1.000000,0.234594,0.044357,0.116880,0.114409,0.258485,...,0.266954,0.263626,0.196160,0.234571,0.214887,0.292585,0.186292,0.145883,0.133066,0.083229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.350861,0.206495,0.170628,0.233341,0.292585,0.225559,0.045502,0.146479,0.162331,0.338573,...,0.423133,0.538736,0.294687,0.361943,0.289467,1.000000,0.221085,0.171633,0.180556,0.098707
146,0.213027,0.141947,0.118172,0.199036,0.186292,0.165081,0.018804,0.075819,0.087233,0.199791,...,0.236376,0.205555,0.153194,0.191219,0.176006,0.221085,1.000000,0.109373,0.101822,0.048725
147,0.173260,0.098200,0.080181,0.129646,0.145883,0.128077,0.034979,0.068281,0.065097,0.173510,...,0.184907,0.158779,0.125657,0.166947,0.127074,0.171633,0.109373,1.000000,0.088012,0.050221
148,0.191958,0.106107,0.081702,0.358112,0.133066,0.104962,0.022511,0.091122,0.090193,0.168091,...,0.193006,0.160427,0.131045,0.200733,0.129238,0.180556,0.101822,0.088012,1.000000,0.720658
